<a href="https://colab.research.google.com/github/bbang3/korean-text-augmentation/blob/klue/experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/korean-text-augmentation

/content/drive/MyDrive/korean-text-augmentation


# Import

In [ ]:
!pip install transformers[pytorch]
!pip install datasets
!pip install accelerate
!pip install evaluate

In [5]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


In [6]:
import os
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm

from transformers import BertModel, BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate

import wandb

In [7]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 81


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
os.environ["WANDB_PROJECT"] = "Korean Text Augmentation - KLUE"

In [9]:
train_path = 'data/train_eda_2.csv'
val_path = 'data/val_low.csv'
test_path = 'data/test.csv'
dataset = load_dataset('csv', data_files={'train': train_path, 'validation': val_path, 'test': test_path})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()
del model

In [10]:
num_labels = 7
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=num_labels).to("cuda")

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def tokenization(example):
    return tokenizer(example["title"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenization, batched=True, remove_columns=["title"])
tokenized_dataset.set_format("torch")

Map:   0%|          | 0/840 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [12]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
training_args = TrainingArguments(
    output_dir='./checkpoints/eda_2',
    num_train_epochs=20,
    warmup_steps=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    seed=42,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    report_to='wandb',
    run_name='eda_2'
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()
wandb.finish()

wandb: Currently logged in as: bbang3. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,1.982900,1.958852,0.157143
2,1.939700,1.974807,0.114286
3,1.899600,1.903685,0.185714
4,1.754300,1.912535,0.242857
5,1.564500,1.898348,0.300000
6,1.388900,1.857504,0.328571
7,1.267300,1.690460,0.414286
8,1.163000,2.001669,0.328571
9,1.075700,1.996086,0.385714
10,1.001100,2.064423,0.328571


eval/accuracy,▂▁▃▄▅▆█▆▇▆▅▆▅▆▆▆▆▆▆▇
eval/loss,▄▄▃▃▃▃▁▄▄▅▆▅▇▇▇█▇███
eval/runtime,▇▁▅▇▂▄█▄▃▄▃▅▅▇▅▃▆▇▅▄
eval/samples_per_second,▂█▄▂▇▅▁▅▆▅▆▄▄▂▄▆▃▂▄▅
eval/steps_per_second,▂█▄▂▇▅▁▅▆▅▆▄▄▂▄▆▃▂▄▅
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,▅██▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▁▁
train/loss,███▇▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


# Load model

In [22]:
num_labels = 7
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
# ft_model = BertForSequenceClassification.from_pretrained(trainer.state.best_model_checkpoint, num_labels=num_labels).to("cuda")
ft_model = BertForSequenceClassification.from_pretrained("checkpoints/eda_2/checkpoint-1060", num_labels=num_labels).to("cuda")

In [23]:
training_args = TrainingArguments(
    output_dir='./checkpoints/tmp',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    seed=42,
    evaluation_strategy='no',
    logging_strategy='epoch',
    save_strategy='no',
    report_to='wandb',
    run_name='evaluation'
)

trainer = Trainer(
    model=ft_model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics
)

In [24]:
trainer.evaluate(eval_dataset=tokenized_dataset['test'])

{'eval_loss': 2.286980390548706,
 'eval_accuracy': 0.36828571428571427,
 'eval_runtime': 65.9933,
 'eval_samples_per_second': 106.071,
 'eval_steps_per_second': 6.637}

In [ ]:
wandb.finish()

In [ ]:
predictions = trainer.predict(test_dataset=tokenized_dataset['validation'])
predictions = np.argmax(predictions.predictions, axis=-1)
predictions

array([3, 3, 2, 3, 2, 1, 1, 3, 2, 1, 3, 1, 3, 1, 3, 3, 3, 3, 1, 3, 3, 1,
       3, 6, 3, 2, 1, 3, 1, 2, 2, 1, 2, 1, 1, 3, 2, 3, 1, 3, 3, 2, 2, 3,
       1, 2, 3, 3, 3, 2, 1, 3, 3, 3, 3, 2, 3, 3, 3, 2, 2, 2, 3, 1, 1, 3,
       1, 3, 3, 6])

In [ ]:
predictions = trainer.predict(test_dataset=tokenized_dataset['test'])
predictions = np.argmax(predictions.predictions, axis=-1)
predictions

KeyboardInterrupt: ignored